# 載入套件
BeautifulSoup、Selenium、time、pandas等等

In [1]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from datetime import datetime
import pandas as pd
import matplotlib.pyplot as plt
import io
import os
import time
import re
import json
import sys
from tqdm.notebook import trange, tqdm #加進度條

# 爬iCheer網站
網址：https://www.icheers.tw/iCheers/Wine

In [106]:
# 商品分類清單獲得
BASE_URL = 'https://www.icheers.tw/iCheers/Wine'
headers = {'user-agent': 'Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/52.0.2743.116 Safari/537.36'}
rsp = requests.get(BASE_URL, headers = headers)
rsp.encoding = 'utf8'
soup = BeautifulSoup(rsp.text, 'html.parser')
category_lst = []
cate_url_lst = []
for menu in soup.findAll('ul',{'id':'Menu_Type','class':'Left_Menu_options'}):
    for li in menu.findAll('li'):
        category = li.getText().replace('\n',"")
        for a in li.findAll('a'):
            cate_url = a.get('href')
            category_lst.append(category)
            cate_url_lst.append(cate_url)
category_lst = category_lst[:-1]
cate_url_lst = cate_url_lst[:-1]
print(category_lst, cate_url_lst)

['紅酒 (10258)', '白酒 (2899)', '粉紅酒 (97)', '氣泡酒 (811)', '甜酒 (752)', '加烈酒 (111)'] ['/iCheers/Wine/WineList/wine_list/1/40/0/T1/0/0/-1/-1/0/0/-1/-1/0/0/0/0/3/0/0/0/0/0', '/iCheers/Wine/WineList/wine_list/1/40/0/T2/0/0/-1/-1/0/0/-1/-1/0/0/0/0/3/0/0/0/0/0', '/iCheers/Wine/WineList/wine_list/1/40/0/T3/0/0/-1/-1/0/0/-1/-1/0/0/0/0/3/0/0/0/0/0', '/iCheers/Wine/WineList/wine_list/1/40/0/T4/0/0/-1/-1/0/0/-1/-1/0/0/0/0/3/0/0/0/0/0', '/iCheers/Wine/WineList/wine_list/1/40/0/T5/0/0/-1/-1/0/0/-1/-1/0/0/0/0/3/0/0/0/0/0', '/iCheers/Wine/WineList/wine_list/1/40/0/T6/0/0/-1/-1/0/0/-1/-1/0/0/0/0/3/0/0/0/0/0']


# 以第一頁為例
1. 網站被JavaScript渲染
2. 網址規則為品項是T1...T6
3. 每個品項頁數是網址wine_list/後面的數字
4. 排除無庫存品項：https://www.icheers.tw//iCheers/Wine/WineList/wine_list/1/40/0/T1/0/0/-1/-1/0/0/-1/-1/0/0/0/0/3/0/0/0/0/0
5. 不排除無庫存品項：https://www.icheers.tw/iCheers/Wine/WineList/wine_list/1/40/0/T1/0/0/-1/-1/0/0/-1/-1/0/0/0/0/3/0/0/0/0/0/0/0/0/0

In [3]:
# 以紅酒第一頁為例

# 排除無庫存品項：https://www.icheers.tw//iCheers/Wine/WineList/wine_list/1/40/0/T1/0/0/-1/-1/0/0/-1/-1/0/0/0/0/3/0/0/0/0/0
# 不排除無庫存品項：https://www.icheers.tw/iCheers/Wine/WineList/wine_list/1/40/0/T1/0/0/-1/-1/0/0/-1/-1/0/0/0/0/3/0/0/0/0/0/0/0/0/0
# 頁數為網址wine_list/後面的數字

import time
tStart = time.time()#計時開始

URL = "https://www.icheers.tw/iCheers/Wine/WineList/wine_list/1/40/0/T2/0/0/-1/-1/0/0/-1/-1/0/0/0/0/3/0/0/0/0/0/0/0/0/0"
chromedriver = '/usr/local/bin/chromedriver' #要用變數指定chromedriver的放置路徑
iCheers = webdriver.Chrome(chromedriver)
#rsp.encoding = 'utf8'
iCheers.get(URL)
soup = BeautifulSoup(iCheers.page_source, 'html.parser')
Name_CH = []
Name_EN = []
Area = []
Year = []
ID = []
URL = []
Award = []
Capacity = []
Discount = []
SellingPrice = []

#第一頁
for item in soup.findAll(name='div', attrs={'class':'ALL'}):
    for ul in item.findAll(name='ul',attrs={'class':'WineInformation'}):
        name_ch = item.find('h1').getText()
        name_en = item.find('h2').getText()
        area = item.find('h3').getText().replace('\n','').replace(' ','')
        lst = ul.findAll(name='li')
        addedid = lst[0].get('addedid')
        year = lst[0].getText()
        url = lst[0].find('a').get('href')
        award = lst[1].getText().replace('\n','')
        capacity = lst[2].getText().replace('\n','')
        discount = lst[3].getText().replace('\n','')
        price = lst[5].getText().replace('\n','')
        Name_CH.append(name_ch)
        Name_EN.append(name_en)
        Area.append(area)
        ID.append(addedid)
        Year.append(year)
        URL.append(url)
        Award.append(award)
        Capacity.append(capacity)
        Discount.append(discount)
        SellingPrice.append(price)

#計時結束
tEnd = time.time()
# #列印結果
tRun = tEnd - tStart
print('run time:%d'%tRun)

run time:9


In [ ]:
### 獲得最後一頁的頁碼
page_index = []
page_url = []
pg = soup.find(name='div', attrs={'class':'Wine_Search_List_Page'})
#print(pg)
for a in pg.findAll('a'):
    pg_idx = a.getText()
    pg_url = a.get('onclick')
    page_index.append(pg_idx)
    page_url.append(pg_url)
print(page_index,page_url)

# 開始跑一種酒類的迴圈
1. 爬下各品項網站的資訊
2. time.sleep()可設定，爬比較多的時候設12秒，比較少的時候設20秒，其實12秒就足夠

In [178]:
# 跑一種酒類迴圈

# 排除無庫存品項：https://www.icheers.tw//iCheers/Wine/WineList/wine_list/1/40/0/T1/0/0/-1/-1/0/0/-1/-1/0/0/0/0/3/0/0/0/0/0
# 不排除無庫存品項：https://www.icheers.tw/iCheers/Wine/WineList/wine_list/1/40/0/T1/0/0/-1/-1/0/0/-1/-1/0/0/0/0/3/0/0/0/0/0/0/0/0/0
# 頁碼為網址wine_list/後面的數字

import time
tStart = time.time()#計時開始


Name_CH = []
Name_EN = []
Area = []
Year = []
ID = []
URL = []
Award = []
Capacity = []
Discount = []
SellingPrice = []


a_URL = 'https://www.icheers.tw/iCheers/Wine/WineList/wine_list/'
b_URL = '/40/0/T6/0/0/-1/-1/0/0/-1/-1/0/0/0/0/3/0/0/0/0/0/0/0/0/0'

chromedriver = '/usr/local/bin/chromedriver' #要用變數指定chromedriver的放置路徑
iCheers = webdriver.Chrome(chromedriver)

try:
    for i in range(1,3,1):
        print('正在爬取：第 %s 頁' % i)
        last_URL = a_URL+str(i)+b_URL
        iCheers.get(last_URL)
        soup = BeautifulSoup(iCheers.page_source, 'html.parser')
        for item in soup.findAll(name='div', attrs={'class':'ALL'}):
            for ul in item.findAll(name='ul',attrs={'class':'WineInformation'}):
                name_ch = item.find('h1').getText()
                name_en = item.find('h2').getText()
                area = item.find('h3').getText().replace('\n','').replace(' ','')
                lst = ul.findAll(name='li')
                addedid = lst[0].get('addedid')
                year = lst[0].getText().replace('\n','')
                url = lst[0].find('a').get('href')
                award = lst[1].getText().replace('\n','')
                capacity = lst[2].getText().replace('\n','')
                discount = lst[3].getText().replace('\n','')
                price = lst[5].getText().replace('\n','')
                Name_CH.append(name_ch)
                Name_EN.append(name_en)
                Area.append(area)
                ID.append(addedid)
                Year.append(year)
                URL.append(url)
                Award.append(award)
                Capacity.append(capacity)
                Discount.append(discount)
                SellingPrice.append(price)
        print('資料總共有 %d 筆' %len(Name_CH))
        time.sleep(20)
except:
    print("哪一頁出錯了"+str(i))
finally:
    iCheers.close()

#計時結束
tEnd = time.time()
# #列印結果
tRun = tEnd - tStart
print('總共爬取時間：%d秒'%tRun)

正在爬取：第 1 頁
資料總共有 60 筆
正在爬取：第 2 頁
資料總共有 104 筆
總共爬取時間：54


In [179]:
c={"Name_CH" : Name_CH,
   "Name_EN" : Name_EN,
   "Area" : Area,
   "Year" : Year,
   "ID" : ID,
   "URL" : URL,
   "Award" : Award,
   "Capacity" : Capacity,
   "Discound" : Discount,
   "SellingPrice" : SellingPrice}
df = pd.DataFrame(c)

#df = pd.DataFrame(df,columns=['Date','Class','Item', 'Price', 'URL'])
print(len(df))

104


In [180]:
df

,Name_CH,Name_EN,Area,Year,ID,URL,Award,Capacity,Discound,SellingPrice
0,赫雷斯．巴桂羅酒莊，1905始祖索雷拉 阿蒙蒂利亞多不甜加烈酒,"Bodegas Pérez Barquero, 1905 Solera Fundaciona...",西班牙＞安達露西亞＞Montilla-Moriles,NV,9592,/iCheers/Wine/WineDetail/wine_detail/9592,RP / 100,750ml,9折,"16,200"
1,赫雷斯．巴桂羅酒莊，1905始祖索雷拉 PX甜型加烈酒,"Bodegas Pérez Barquero, 1905 Solera Fundaciona...",西班牙＞安達露西亞＞Montilla-Moriles,NV,9593,/iCheers/Wine/WineDetail/wine_detail/9593,RP / 99,750ml,9.5折,"17,000"
2,盧世濤雪莉酒莊，羅斯埃克羅 阿蒙提亞多雪莉酒,"Bodegas Lustau, Amontillado Los Arcos",西班牙＞安達露西亞＞Jerez-Xérès-Sherry,NV,8216,/iCheers/Wine/WineDetail/wine_detail/8216,2015,375ml,7折,430
3,盧世濤雪莉酒莊，聖艾密里歐 PX甜型雪莉酒,"Bodegas Lustau, Pedro Ximénez San Emilio",西班牙＞安達露西亞＞Jerez-Xérès-Sherry,NV,8223,/iCheers/Wine/WineDetail/wine_detail/8223,2015,375ml,7.1折,620
4,禾富酒廠，紅牌系列 珍藏波特酒,"Wolf Blass , Red Label Reserve Tawny",澳洲＞南澳,NV,8076,/iCheers/Wine/WineDetail/wine_detail/8076,BC / 89,750ml,9折,"1,080"
...,...,...,...,...,...,...,...,...,...,...
99,道斯波特酒莊，聖荷拉里貝拉單一園 年份波特酒,"Dow's Port, Quinta Senhora da Ribeira Vintage ...",葡萄牙＞波特與斗羅河谷,2013,20668,/iCheers/Wine/WineDetail/wine_detail/20668,WS / 94,750ml,,"2,200"
100,道斯波特酒莊，聖荷拉里貝拉單一園 年份波特酒,"Dow's Port, Quinta Senhora da Ribeira Vintage ...",葡萄牙＞波特與斗羅河谷,2015,20669,/iCheers/Wine/WineDetail/wine_detail/20669,WE / 93,750ml,,"2,200"
101,夸蒂酒莊，極樂天堂蜜思嘉甜紅酒,"Quady Winery, Elysium Muscat Sweet Red Wine",美國＞加州,2016,17591,/iCheers/Wine/WineDetail/wine_detail/17591,,375ml,8.4折,500
102,葛蕾絲酒莊，周五郎馬德拉式加烈甜酒,"Grace Wine, Koshu Fortified Wine",日本＞甲信越地方＞山梨県,NV,5349,/iCheers/Wine/WineDetail/wine_detail/5349,,720ml,9折,"1,320"


In [181]:
# 第一次存新的檔案
df.to_csv('./iCheers_加烈酒.csv', index = False)

# 資料合併
將下載下來的資料合併，取出編號list來爬。

In [4]:
from glob import glob
 
files = glob('iCheers_data/iCheers_*.csv')
print(files)

df = pd.concat(
    (pd.read_csv(file, usecols=['Name_CH','Name_EN','ID'], dtype={'Name_CH': str,'Name_EN': str, 'ID':int}) for file in files), ignore_index=True)

['iCheers_data/iCheers_紅酒.csv', 'iCheers_data/iCheers_氣泡酒.csv', 'iCheers_data/iCheers_加烈酒.csv', 'iCheers_data/iCheers_白酒.csv', 'iCheers_data/iCheers_甜酒.csv', 'iCheers_data/iCheers_粉紅酒.csv']


In [5]:
ID_lst = df['ID'].tolist()
print(len(ID_lst))

13641


In [6]:
ID_lst

[17132,
 13488,
 1642,
 1580,
 12678,
 12679,
 12680,
 12681,
 12682,
 22038,
 17537,
 180,
 5963,
 7807,
 179,
 8085,
 15385,
 17680,
 20301,
 15384,
 18570,
 15439,
 13652,
 12461,
 9035,
 17270,
 14797,
 12127,
 11888,
 17548,
 20699,
 9367,
 9220,
 7491,
 1026,
 1027,
 1037,
 1038,
 1039,
 8381,
 5787,
 10187,
 16720,
 12686,
 11039,
 11634,
 6225,
 17536,
 6591,
 5383,
 5384,
 8703,
 7543,
 20811,
 6931,
 6636,
 5266,
 18059,
 1643,
 8103,
 18571,
 5219,
 7365,
 6749,
 6748,
 12659,
 12660,
 12661,
 12662,
 12663,
 12664,
 14234,
 14235,
 6747,
 1343,
 2141,
 144,
 17412,
 18678,
 18677,
 17216,
 15771,
 18539,
 8687,
 14499,
 1261,
 15752,
 7972,
 12707,
 621,
 14241,
 21632,
 15169,
 13709,
 17627,
 7233,
 7234,
 11336,
 6301,
 5223,
 11949,
 12684,
 5889,
 11581,
 17314,
 9415,
 104,
 12683,
 17133,
 1341,
 17538,
 20477,
 20478,
 20475,
 20476,
 7366,
 19538,
 20471,
 11569,
 12960,
 10537,
 17210,
 16695,
 1040,
 1041,
 1042,
 1030,
 16356,
 17621,
 13587,
 8733,
 4950,
 1605

# 根據編號爬下每個酒的資訊
1. 網址：https://www.icheers.tw/iCheers/Wine/WineDetail/wine_detail/17132
2. 網址後面為編號
3. 沒有被JavaScript渲染
4. 依據ID_lst，用迴圈抓下每個酒的資訊
5. 欄位包括編號、中文名字、英文名字、容量、定價、酒窩價、產區、法定等級、酒莊、類型、品種、酒款介紹、品飲筆記

In [46]:
# 跑一種酒
BASE_URL = 'https://www.icheers.tw/iCheers/Wine/WineDetail/wine_detail/17132'
headers = {'user-agent': 'Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/52.0.2743.116 Safari/537.36'}
rsp = requests.get(BASE_URL, headers = headers)
rsp.encoding = 'utf8'
soup = BeautifulSoup(rsp.text, 'html.parser')

ID = []
Name_CH = []
Name_EN = []
Capacity = []
OriginalPrice = []
SellingPrice = []
Area = []
Level = []
Winery = []
Flavor = []
Grape = []
Intro = []
Note = []

for menu in soup.findAll('div',{'class':'wine_detail_wineinfo_layout'}):
    name_ch = menu.find('h2').getText().replace(' ','')
    name_en = menu.find('h1').getText().replace('\n','').replace(' ','')
    for ul in menu.findAll('ul',{'class':'Detail'}):
        lst = ul.findAll(name='li')
        bar0 = lst[0].getText().replace('#','').replace('\n','')
        if bar0 == str(17132):
            ori_price = lst[1].find('strong',{'class':'wine_detail_wineinfo_OrginalPrice'}).getText()
            sell_price = lst[1].find('span').getText().replace('\n',' ')
            area = lst[2].find('em',{'class':'Info'}).getText().replace('\n','')
            level = lst[3].find('em',{'class':'Info'}).getText().replace('\n','')
            winery = lst[4].find('em',{'class':'Info'}).getText().replace('\n','')
            flavor = lst[5].find('em',{'class':'Info'}).getText().replace('\n','')
            grape = lst[6].find('em',{'class':'Info'}).getText().replace(' ','').replace('，','')
        else:
            ori_price = lst[2].find('strong',{'class':'wine_detail_wineinfo_OrginalPrice'}).getText()
            sell_price = lst[2].find('span').getText().replace('\n',' ')
            area = lst[3].find('em',{'class':'Info'}).getText().replace('\n','')
            level = lst[4].find('em',{'class':'Info'}).getText().replace('\n','')
            winery = lst[5].find('em',{'class':'Info'}).getText().replace('\n','')
            flavor = lst[6].find('em',{'class':'Info'}).getText().replace('\n','')
            grape = lst[7].find('em',{'class':'Info'}).getText().replace(' ','').replace('，','')
        
for text in soup.findAll('div',{'class':'Part'}):
    h3 = text.find('h3').getText()
    if h3 == '酒款介紹：':
        intro = text.find('p').getText().replace(' ','').replace('\n','')
    if h3 == '品飲筆記：':
        note = text.find('p').getText().replace(' ','').replace('\n','')

print(f"定價：{ori_price}\n酒窩價：{sell_price}\n產區：{area}\n法定等級：{level}\n酒莊：{winery}\n類型：{flavor}\n品種：{grape}\n酒款介紹：{intro}\n")
print(f"品飲筆記：{note}")

定價：$ 2,100
酒窩價： $ 1,499 
產區：法國／隆河谷／Châteauneuf-du-Pape／
法定等級：法定產區管制（AC）
酒莊：Maison de Grand Esprit
類型：紅葡萄酒／干型（不甜）
品種：
80%
Grenache
10%
Mourvèdre
6%
Syrah
4%
Cinsault

酒款介紹：20%果實成串低溫發酵，以維持清新果香，為期5週的泡皮賦予濃縮飽滿的酒體。發酵完成後，20%酒液培養於法國橡木桶，其餘則置於不鏽鋼槽。 深沉石榴紅酒澤，香氣奔放集中，香料、深色水果果醬甜香，些許甘草、八角及紅椒香料感，讓人聯想到溫暖與陽光普照的夏天。口感飽滿有結構，充滿各式香料、紫羅蘭風味，尾韻帶出黑莓調性，為長期窖藏首選。建議可與牛肋排、烤鴿佐牛肝菌與乳酪等料理搭配。 

整體而言，2015年的氣候炎熱乾燥，生長季偶有雷雨。日照充足，超過六個月的溫暖陽光，讓葡萄能完美成熟。


# 寫迴圈
1. 發現每個酒品的欄位不太一致，若用lst的index去抓的話，可能會有錯誤
2. 因此，找到所有可能的欄位，包括產區、法定等級、酒莊、類型、品種、建議適飲期
3. 先讓每個欄位的value=NA，再用elseif函數填充進去，沒有填到的值則為NA
4. 利用try使得遇到error不會停止迴圈
5. 每次迴圈就建立一個DataFrame，每次都合併，最後結果叫做df
6. 找出了四個不同的網站 #ID_try = ['17132','18570','8703','18539']

In [15]:
# 檢查(或刪減)ID_lst
# for i in range(500):
#     ID_lst.pop(0)
print(len(ID_lst))
print(ID_lst[976])

13641
20567


In [8]:
# 迴圈跑ID_lst的每個ID

# ID_try = ['17132','18570','8703','18539','6098']
c = {}
df = pd.DataFrame()
headers = {'user-agent': 'Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/52.0.2743.116 Safari/537.36'}
a_URL = 'https://www.icheers.tw/iCheers/Wine/WineDetail/wine_detail/'
for i in trange(9025,13641,1):
    URL = a_URL+str(ID_lst[i])
    rsp = requests.get(URL, headers = headers)
    rsp.encoding = 'utf8'
    soup = BeautifulSoup(rsp.text, 'html.parser')
    ID = 'NA'
    name_ch = 'NA'
    name_en = 'NA'
    ori_price = 'NA'
    sell_price = 'NA'
    area = 'NA'
    level = 'NA'
    winery = 'NA'
    flavor = 'NA'
    grape = 'NA'
    period = 'NA'
    ID = ID_lst[i]
    for menu in soup.findAll('div',{'class':'wine_detail_wineinfo_layout'}):
        name_ch = menu.find('h2').getText().replace(' ','').replace('\n','').replace('\r','')
        name_en = menu.find('h1').getText().replace('  ','').replace('\n','').replace('\r','')
        for ul in menu.findAll('ul',{'class':'Detail'}):
            lst = ul.findAll(name='li')
            bar0 = lst[0].getText().replace('#','').replace('\n','')
            try:
                if bar0 == str(ID_lst[i]):
                    ori_price = lst[1].find('strong',{'class':'wine_detail_wineinfo_OrginalPrice'}).getText()
                    sell_price = lst[1].find('span').getText().replace('\n',' ')
                else:
                    ori_price = lst[2].find('strong',{'class':'wine_detail_wineinfo_OrginalPrice'}).getText()
                    sell_price = lst[2].find('span').getText().replace('\n',' ')
            except:
                print('ID # %s 價格不存在(NA)' %ID_lst[i])
            try:
                em = ul.findAll('em')
                for j in range(0,11,2):
                    key = em[j].getText().replace('：','')
                    if key == '產區':
                        area = em[j+1].getText().replace('\n','')
                    elif key == '法定等級':
                        level = em[j+1].getText().replace('\n','')
                    elif key == '酒莊':
                        winery = em[j+1].getText().replace('\n','')
                    elif key == '類型':
                        flavor = em[j+1].getText().replace('\n','')
                    elif key == '品種':
                        grape = em[j+1].getText().replace('\n','').replace('\r','')
                    elif key == '建議適飲期':
                        period = em[j+1].getText().replace('\n','')
            except:
                print('ID # %s 有欄位產生缺值(NA)' %ID_lst[i])
    c = dict(ID=ID, Name_CH=name_ch, Name_EN=name_en, OriginalPrice=ori_price, SellingPrice=sell_price,
             Area=area, Level=level, Winery=winery, Flavor=flavor, Grape=grape, Period=period)
    df1 = pd.DataFrame(c, index=[0])
    df = pd.concat([df, df1], ignore_index=False)
#     if i == ID_lst[500]:
#         break

ID # 17132 有欄位產生缺值(NA)
ID # 18570 有欄位產生缺值(NA)
ID # 18539 有欄位產生缺值(NA)
ID # 6098 價格不存在(NA)



In [9]:
df

,ID,Name_CH,Name_EN,OriginalPrice,SellingPrice,Area,Level,Winery,Flavor,Grape,Period
0,17132,艾斯莊園．奧秘系列教皇新堡紅酒,"Maison de Grand Esprit, La Mystèriale Châteaun...","$ 2,100","$ 1,499",法國／隆河谷／Châteauneuf-du-Pape／,法定產區管制（AC）,Maison de Grand Esprit,紅葡萄酒／干型（不甜）,80 % ...,NA
0,18570,弗莫伊酒莊．莊園希哈紅酒,"Fermoy Estate, Fe Shiraz 2014","$ 1,600","$ 1,440",澳洲／西澳／Margaret River／,NA,Fermoy Estate,紅葡萄酒／干型（不甜）,95 % ...,現在適飲
0,8703,雲端酒莊．雲端紅酒,"Au Sommet, Au Sommet 2013","$ 13,500","$ 12,500",美國／加州／,美國葡萄酒法定產區（AVA）,Au Sommet,紅葡萄酒／干型（不甜）,97 % ...,2015-2030
0,18539,鹿躍酒莊．費園卡本內蘇維濃紅酒2013-20156入垂直套組,"Stag's Leap Wine Cellars, FAY Cab 2013-2015 6-...","$ 37,200","$ 25,800",美國／加州／North Coast／Stags Leap District／,NA,Stag's Leap Wine Cellars,紅葡萄酒／干型（不甜）,Cabernet Sauvignon，,現在適飲
0,6098,葛利拉侯克堡．葛利拉侯克一軍紅酒,"Château Gree Laroque, Château Gree Laroque 2010",NA,NA,法國／波爾多／,優級波爾多,Château Gree Laroque,紅葡萄酒／干型（不甜）,90 % ...,2016-2024


In [32]:
# 第一次存新的檔案
df.to_csv('./iCheers_data/酒品詳細資料.csv', index = False)
print(len(df))

501


In [8]:
#之前存過一個CSV檔，想把之後爬的直接接在之前資料的後面
df.to_csv('./iCheers_data/酒品詳細資料.csv', index=False, mode='a', header=False)
print(len(df))

4616


In [9]:
# look up updates
k = pd.read_csv('./iCheers_data/酒品詳細資料.csv')
k

,ID,Name_CH,Name_EN,OriginalPrice,SellingPrice,Area,Level,Winery,Flavor,Grape,Period
0,17132,艾斯莊園．奧秘系列教皇新堡紅酒,"Maison de Grand Esprit, La Mystèriale Châteaun...","$ 2,100","$ 1,499",法國／隆河谷／Châteauneuf-du-Pape／,法定產區管制（AC）,Maison de Grand Esprit,紅葡萄酒／干型（不甜）,80 % ...,NaN
1,13488,亨利瑞伯赫蘇酒莊．「馬立．香貝丹」特級園紅酒,"Domaine Henri Rebourseau, ""Mazy-Chambertin"" Gr...","$ 8,870","$ 7,980",法國／布根地／Côte de Nuits／Gevrey-Chambertin／,特級園,Domaine Henri Rebourseau,紅葡萄酒／干型（不甜）,100 % ...,現在適飲
2,1642,路易士酒莊．那帕谷卡本內‧蘇維濃紅酒,"Lewis Cellars , Napa Valley Cabernet Sauvignon...","$ 4,160","$ 3,661",美國／加州／North Coast／Napa Valley／,美國葡萄酒法定產區（AVA）,Lewis Cellars,紅葡萄酒／干型（不甜）,100 % ...,2014-2026
3,1580,巴肯酒莊．典藏希哈紅酒,"Barkan Winery (יקבי ברקן), Reserve Shiraz 2010","$ 1,530","$ 1,377",以色列／戈蘭高地／הגליל／,NaN,Barkan Winery (יקבי ברקן),紅葡萄酒／干型（不甜）,100 % ...,現在適飲
4,12678,積架酒莊．翁普依莊園經典紅酒,"E. Guigal, Château d'Ampuis 1999","$ 8,200","$ 7,380",法國／隆河谷／Côte Rôtie／,法定產區管制（AC）,E. Guigal,紅葡萄酒／干型（不甜）,，,現在適飲
...,...,...,...,...,...,...,...,...,...,...,...
13637,26958,艾斯莊園．綺幻系列普羅旺斯粉紅酒,"Maison de Grand Esprit, L'Être Magique Côtes d...","$ 1,180",$ 850,法國／普羅旺斯／Côtes de Provence／,法定產區管制（AC）,Maison de Grand Esprit,粉紅葡萄酒／干型（不甜）,55 % ...,NaN
13638,27388,格雷西酒莊．埃特納火山粉紅酒,"Graci, Etna Rosato 2018","$ 1,200","$ 1,080",義大利／西西里／Etna／,法定產區（DOC）,Graci,粉紅葡萄酒／干型（不甜）,Nerello Mascalese，,NaN
13639,13913,羅亞爾合作協會．安茹粉紅酒,"Alliance Loire, Rose d'Anjou Villa Lorane 2016",$ 705,$ 640,法國／羅亞爾河谷／Anjou／,法定產區管制（AC）,Alliance Loire,粉紅葡萄酒／半干型,Cabernet Franc，Gamay，Grolleau，,現在適飲
13640,25273,羅亞爾合作協會．安茹粉紅酒,"Alliance Loire, Rose d'Anjou Villa Lorane 2018",$ 700,$ 630,法國／羅亞爾河谷／Anjou／,法定產區管制（AC）,Alliance Loire,粉紅葡萄酒／半干型,，,現在適飲
